In [1]:
import pandas as pd
import os
import sys
import json
import re

In [2]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'GoEmotions-pytorch') )
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )


In [3]:
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
import re
from tqdm import tqdm
import numpy as np 
import pandas as pd
import pickle

In [35]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )

# Define data output path
transcript_path = os.path.join(git_dir, 'data', 'pm-transcripts-processed/aggregated-spellfixed-typed.csv')
df = pd.read_csv(transcript_path)

In [5]:
# df['content'] = df['content'].str.replace(u'\xa0', u' ')
# df['content'] = df['content'].str.replace(' \]\]\>', '')

In [36]:
df.head()

,Unnamed: 0,pm,date,type,content,sentences
0,0,"Abbott, Tony",24/08/2014,Media Release,Every day the Government is working to build a...,['every day the government is working to build...
1,1,"Gorton, John",14/03/1968,Statement in Parliament,"STATEMENT BY THE PRIME MINISTER, THE RT. HON....",['the house of representatives on a of march o...
2,2,"Turnbull, Malcolm",17/11/2016,Transcript,PRIME MINISTER: This is a great initiative. Wh...,['what you re seeing here is real enthusiasm f...
3,3,"Fraser, Malcolm",27/09/1978,Media Release,"FOR PRESS 27 SEPTEMBER, 1978 GRANT TO WORLD W...",['for press of september of of grant to world ...
4,4,"Howard, John",24/03/2004,Interview,"JOURNALIST: Prime Minister, is Mark Latham pla...",['journalist prime minister is mark latham pla...


In [59]:
df_speeches = df[((df['type']=='Speech') | (df['type']=='Media Release'))]

In [63]:
def batch_gen(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [77]:
tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = model.to('cuda:0')

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3,
    device=0
)


processed_sentence_lists = []
for ind, sentence_list_repr in enumerate(tqdm(df_speeches['sentences'])):
    processed_sentences = []
    
    sentence_list_raw = eval(sentence_list_repr)
    sentence_list = []
    
    for sentence_ind, sentence in enumerate(sentence_list_raw):
        tokens = tokenizer(sentence)
        if len(tokens['input_ids'])>500:
            print('Sentence %d in document %d is too long'%(ind, sentence_ind))
        else:
            sentence_list.append(sentence)
    
    for batch in batch_gen(sentence_list, n=32):
        processed_sentences.extend(goemotions(batch))
        #processed_sentences.append(goemotions(sentence)[0])
            #ts['emotions'].append(goemotions(sentence)[0])
    processed_sentence_lists.append(processed_sentences)
        

  7%|▋         | 910/13674 [01:05<17:24, 12.22it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors


Sentence 911 in document 23 is too long


 11%|█         | 1513/13674 [01:49<17:46, 11.40it/s]

Sentence 1512 in document 20 is too long


 11%|█         | 1528/13674 [01:51<21:13,  9.54it/s]

...

 98%|█████████▊| 13454/13674 [16:56<00:10, 20.86it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (672 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (832 > 512). Running this sequence through the model will result in indexing errors


Sentence 13454 in document 28 is too long
Sentence 13454 in document 30 is too long


100%|█████████▉| 13666/13674 [17:12<00:00, 20.04it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors


Sentence 13667 in document 16 is too long


100%|██████████| 13674/13674 [17:12<00:00, 13.24it/s]


In [82]:
processed_sentence_lists[1000]

[{'labels': ['neutral'], 'scores': [0.99498135]},
 {'labels': ['neutral'], 'scores': [0.9990583]},
 {'labels': ['caring', 'desire'], 'scores': [0.9703579, 0.360673]},
 {'labels': ['admiration', 'neutral'], 'scores': [0.40888488, 0.991501]},
 {'labels': ['joy'], 'scores': [0.9819531]},
 {'labels': ['desire', 'optimism'], 'scores': [0.4751356, 0.95900327]}]

In [79]:
len(df_speeches)

13674

In [67]:
eval(df_speeches['sentences'][0])

['every day the government is working to build a stronger economy so that we can help australians families and small businesses',
 'a stronger economy means more investment more exports and more jobs',
 'now with the return of parliament on tuesday the government is continuing to put in place the policies that are part of our economic action strategy',
 'these policies are already making it easier for businesses to compete and to employ more people',
 'in the last week of parliament we scrapped the carbon tax and the benefits are already filtering through were working with the senate to scrap the mining tax a tax that is raised little yet costs much',
 'we will cut company tax making it easier for many small businesses and our record of billion infrastructure programme is underway',
 'this programme means that major works are happening in our cities and regional areas',
 'and because jobs depend on major investments were clearing the backlog of environmental approvals with approvals al

In [89]:
df_speeches_processed = df_speeches.copy()

In [90]:
df_speeches_processed['sentences_emotions'] = processed_sentence_lists
#processed_sentence_lists[0]

In [91]:
df_speeches_processed.head()

,Unnamed: 0,pm,date,type,content,sentences,sentences_emotions
0,0,"Abbott, Tony",24/08/2014,Media Release,Every day the Government is working to build a...,['every day the government is working to build...,"[{'labels': ['approval', 'optimism', 'realizat..."
3,3,"Fraser, Malcolm",27/09/1978,Media Release,"FOR PRESS 27 SEPTEMBER, 1978 GRANT TO WORLD W...",['for press of september of of grant to world ...,"[{'labels': ['neutral'], 'scores': [0.9980432]..."
8,8,"Fraser, Malcolm",21/07/1976,Media Release,FOR PRESS 21 July 1976 SENATOR COTTON TO LEAD...,['for press of july of of senator cotton to le...,"[{'labels': ['neutral'], 'scores': [0.9990237]..."
9,9,"Gillard, Julia",25/08/2011,Media Release,Patients and families in regional communities...,['patients and families in regional communitie...,"[{'labels': ['approval', 'neutral'], 'scores':..."
12,12,"Hawke, Robert",06/06/1986,Media Release,PRIMEST AINIE FOR MEDIA 6 JUNE 1986 I am plea...,['priest annie for media a june of of i am ple...,"[{'labels': ['neutral'], 'scores': [0.9751662]..."


In [97]:
# Convert string repr of sentences dicts back into dicts
df_speeches_processed['sentences'] = df_speeches_processed['sentences'].apply(lambda x: eval(x))

In [100]:
df_speeches_processed.to_pickle( os.path.join(git_dir, 'data', 'pm-transcripts-processed/aggregated-spellfixed-typed-emotions.pkl') )